In [ ]:
import os
import sys
import json
import warnings
import traceback

import numpy as np
from scipy import signal
import librosa

sys.path.insert(0, '/mnt/d/projects/bassline_extraction') 

from utilities import * #as utils
from plotting import *
from transcription import *
from signal_processing import *

warnings.filterwarnings('ignore')
%matplotlib inline
np.set_printoptions(suppress=True)

import importlib

# Initialization

## Directories

**TO DO: aligned_beat_positions => progression_beat_positions**

In [ ]:
with open('../data/ouz_tracks.txt', 'r') as infile:
    track_titles = infile.read().split('\n')
       
_,_, bad_examples = next(os.walk('../data/bassline_extraction/beat_grid/bad_examples'))
bad_examples = [title.split('.txt')[0] for title in bad_examples]
       
with open('../data/metadata/scales_frequencies.json','r') as infile:
    scales = json.load(infile)

with open('../data/metadata/TechHouse_track_dicts.json','r') as infile:
    track_dicts = json.load(infile) 

importlib.reload(ts)

In [ ]:
exceptions = []

for title in track_titles:

    try:

        print('{}\n'.format(title))
        BPM = float(track_dicts[title]['BPM'])
        beat_length = 60/BPM
        
        quarter_beat_positions = get_quarter_beat_positions(get_progression_beat_positions(title))

        if title in bad_examples:
            with open(os.path.join('../data/bassline_extraction/beat_grid/bad_examples',title+'.txt'), 'r') as infile:
                dpqb = float(infile.read())

            #if dpqb > 
                #continue
            print("Beat grid problematic! dpqb: {:.2f}%\n{}".format(dpqb,title))

        notes, scale_frequencies, out_notes, out_frequencies = get_track_scale(title, track_dicts, scales)

        fs = 44100
        chorus, bassline, _ = load_audio(title) # read the chorus and the bassline

        # Spectrogram 
        beat_factor = 8
        win_length = int((beat_length/beat_factor)*fs) 
        n_fft = 4096*8
        hop_length = int(win_length/2) 

        bassline_spectrogram = extract_dB_spectrogram(bassline, n_fft, win_length, hop_length)

        # F0 estimation 
        beat_factor = 8 # quarter 4, 1/8th 8...
        F0_estimate, pitch_track = pYIN_F0(bassline,
                                           fs,
                                           int((beat_length/beat_factor)*fs),
                                           threshold=0.05)     
        
        pitch_track_quantized = adaptive_voiced_region_quantization(pitch_track, 
                                                                    quarter_beat_positions, 
                                                                    scale_frequencies, 
                                                                    epsilon=2)

        # Extract the Notes
        bassline_notes = extract_notes(pitch_track_quantized, notes, scale_frequencies, out_notes, out_frequencies)

        plot_note_comparison(title, bassline_spectrogram, fs, hop_length, F0_estimate, bassline_notes, save=True, plot_title='AQ_UNK_found')
            
    except KeyboardInterrupt:
        import sys
        sys.exit()
        pass 
    except KeyError:
        print("Track doesn't exist in track_dicts: {}\n".format(title))
    except Exception as ex:
        exception_str = ''.join(traceback.format_exception(etype=type(ex), value=ex, tb=ex.__traceback__))
        print(exception_str+'\n')
        exceptions.append(exception_str)